In [48]:
import numpy as np
import pandas as pd

In [49]:
df = pd.read_excel('FilteredViewTransactionsExpensesAll_CSILakiMIesNet_Eng.xlsx')
df.head(2)
df['Assigment name '] = df['Assignment / Toimeksianto'].map(lambda x: ' '.join(x.split('-')[1:]))

In [50]:
date_columns = ['Date / Päiväys', 'Entry Date / Kirjauspäivä', 'Invoicing Date / Laskutuspäivä', 'Payment Date / Maksupäivä']

df[date_columns] = df[date_columns].astype(str)
for date_column in date_columns:
    df[date_column] = df[date_column].apply(lambda x: str(x)[:10])

new_columns = [column.split('/')[0][:-1] for column in df.columns]

df.columns = new_columns

bool_columns = ['Internal', 'Non-Billable']

for colname in bool_columns:
    df[colname] = df[colname].apply(lambda x: 'True' if x else 'False')

df['Assignment'] = df['Assignment'].apply(lambda x: x.split()[0])

In [51]:
non_use = ['Business Unit', 'Business Area', 'Job Title', 'Manager', 'Account', 'Assignment.agency_gui',
           'Assignment.Owner_Gui', 'Assignment.Responsible_Person_Gui','Assignment.Record_Gui', 'User',
           'User.Record_Gui', 'Users.Manager_Gui', 'Registered']

common_cols =  ['Parent Company', 'Parent Company Type', 'Parent Company Home Country',
       'Parent Company Industry', 'Parent Company Responsible', 'Principal',
       'Principal Type', 'Principal Home Country', 'Payer',
       'Payer Home Country', 'Industry', 'Customer Responsible',
       'Seller', 'Cost Center', 'Assignment Type', 'Author', 'Category', 
        'Employee Role', 'Internal', 'Assigment name', 'Customer Responsible', 'Owner', 'Responsible']

df = df[[x for x in df.columns if x not in non_use]]

uncommon_cols = [col for col in df.columns if col not in common_cols + ['Assignment', 'Transaction Type']]

In [52]:
df_common = df[['Assignment'] + common_cols].drop_duplicates(['Assignment']).set_index('Assignment')

In [53]:
rename_dict = {
    'Transaction Type Englis': 'Transaction Type English',
    'Transaction status (Ope': 'Transaction status',
    'Status (Registere': 'Status',
    'Type (Fe': 'Type',
}

def aggregator(rec):
#     print(rec[['Billable Hours', 'Status (Registere', 'Paid', 'Invoiced', 'Margin', 'Invoicing Date', 'Total']])
    max_index = np.max(list(rec.index))
    result = pd.Series(rec.loc[max_index, :])
    result['Billable Hours'] = np.sum(rec['Billable Hours'].dropna())
    result['Hours Worked'] = np.sum(rec['Hours Worked'].dropna())
    result['Paid'] = np.sum(rec['Paid'].dropna())
    result['Invoiced'] = np.max(rec['Invoiced'].dropna())
    result['Margin'] = np.max(rec['Margin'].dropna())
    result['Total'] = np.max(rec['Total'].dropna())
    result_new = result[uncommon_cols]
    result_new = result_new.rename(rename_dict)
    return result_new.to_dict()

In [54]:
result = df.groupby(['Assignment', 'Transaction'], as_index=True).apply(lambda x: aggregator(x))
result = result.reset_index()
result = result.rename(columns = {0: 'Transactions'})

In [55]:
def transaction_num_adder(rec):
#     print(rec)
#     new_rec = pd.DataFrame(rec)
#     new_rec.loc[:,'transaction_num'] = np.arange(len(new_rec))
#     print(new_rec)
#     new_rec.T.to_dict()
#     print(rec)
#     try:
        for transaction_num, index in enumerate(rec.index):
            rec.loc[index, 'Transactions'].update(
                {
                    'Transaction Number': transaction_num + 1,
                }
            )
#     except:
#         print(rec)
#     return new_rec.T.to_dict()

In [56]:
result.groupby(['Assignment'], as_index = False).agg(transaction_num_adder)

,Assignment,Transaction,Transactions
0,1,None,None
1,10,None,None
2,11,None,None
3,12,None,None
4,13,None,None
5,14,None,None
6,15,None,None
7,16,None,None
8,17,None,None
9,18,None,None


In [57]:
# result.loc[result['Assignment'] == '12', 'Transactions'].iloc[2]

In [58]:
result_json = {}
def anton_agg(rec):
    key = rec['Assignment'].iloc[0]
    result = {rec.loc[i,'Transaction']: rec.loc[i,'Transactions'] for i in rec.index}
    output = {
        key: result,
    }
    result_json.update(output)

after_group_by = result.groupby(['Assignment'], as_index=False).agg(anton_agg)

series_uncommon = pd.Series(result_json, name = 'Transactions')

In [59]:
df_prefinal = pd.concat([df_common, series_uncommon], axis = 1)
df_prefinal = df_prefinal.rename(columns = {0: 'Transactions'})

sum_columns = ['Total', 'Margin', 'Paid', 'Invoiced', 'Billable Hours', 'Hours Worked']

In [60]:
df_prefinal;

In [61]:
for colname in sum_columns:
    df_prefinal[colname + '_sum'] = df_prefinal['Transactions'].map(lambda x: sum([x[j][colname] for j in list(x.keys())]))

In [62]:
df_prefinal.to_csv('df_prefinal.csv', index = False)

In [63]:
common_dict = df_prefinal.drop(['Transactions'], axis = 1).T.to_dict()

In [64]:
series_common = pd.Series(common_dict, name= 'Info')

In [121]:
group_by_min = df.groupby(['Assignment']).agg(
    {
        'Entry Date': np.min,
    }
)
group_by_min.columns = ['min_date']
group_by_min['min_date'] = pd.to_datetime(group_by_min['min_date'])

group_by_max = df.groupby(['Assignment']).agg(
    {
        'Entry Date': np.max,
    }
)
group_by_max.columns = ['max_date']
group_by_max['max_date'] = pd.to_datetime(group_by_max['max_date'])


df_time = pd.concat([group_by_max, group_by_min], axis = 1)
df_time['date_sample'] = df_time.apply(lambda x: str(((x['max_date']- x['min_date']) // 3) * 2 + x['min_date'])[:10], axis=1).astype(str)#[:10]
df_time['date_more'] = df_time.apply(lambda x: str(x['max_date'] + pd.to_timedelta(1,unit='D'))[:10], axis=1)
df_time['min_date'] = df_time['min_date'].apply(lambda x: str(x)[:10])
df_time['max_date'] = df_time['max_date'].apply(lambda x: str(x)[:10])

# df_time['Info'] = df.apply(lambda x: x.to_dict(), axis = 1)

In [122]:
# series_dates =  df.apply(lambda x: x.to_dict(), axis = 1)
# series_dates = series_dates.rename(columns = {0:'Info'})
# series_dates.name = 'Info'

In [129]:
json_4 = pd.concat([series_common, series_uncommon, df_time], axis=1).T.to_dict()

In [133]:
# json_4['1']['date_sample']

In [131]:
import json

In [132]:
json.dump(json_4, open('json_4', 'w'))

In [22]:
def aggregator_transactions(rec):
#     print(rec[['Billable Hours', 'Status (Registere', 'Paid', 'Invoiced', 'Margin', 'Invoicing Date', 'Total']])
    max_index = np.max(list(rec.index))
    result = pd.Series(rec.loc[max_index, :])
    result['Billable Hours'] = np.sum(rec['Billable Hours'].dropna())
    result['Hours Worked'] = np.sum(rec['Hours Worked'].dropna())
    result['Paid'] = np.sum(rec['Paid'].dropna())
    result['Invoiced'] = np.max(rec['Invoiced'].dropna())
    result['Margin'] = np.max(rec['Margin'].dropna())
    result['Total'] = np.max(rec['Total'].dropna())
    return result.to_dict()

In [23]:
df_transactions = df.groupby(['Assignment', 'Transaction'], as_index=True).apply(lambda x: aggregator_transactions(x))
df_transactions = df_transactions.reset_index()
df_transactions = df_transactions.rename(columns = {0: 'Transactions'})

In [24]:
content = df_transactions['Transactions'].apply(lambda x: pd.Series(x))

In [25]:
content['Hour Price']

0       170.0
1       170.0
2         NaN
3       170.0
4       170.0
5       170.0
6       170.0
7         NaN
8       170.0
9         NaN
10        0.0
11      170.0
12      170.0
13      170.0
14        NaN
15        NaN
16      100.0
17      170.0
18      200.0
19      200.0
20      170.0
21        NaN
22      170.0
23        NaN
24        NaN
25      170.0
26      170.0
27        NaN
28        NaN
29      170.0
        ...  
1130    170.0
1131    170.0
1132      NaN
1133      NaN
1134      NaN
1135      NaN
1136    170.0
1137    170.0
1138      0.0
1139    170.0
1140    170.0
1141    170.0
1142    170.0
1143    170.0
1144    170.0
1145    170.0
1146    170.0
1147    170.0
1148      NaN
1149    170.0
1150    170.0
1151    170.0
1152    250.0
1153    170.0
1154    150.0
1155      NaN
1156    170.0
1157    170.0
1158    500.0
1159    170.0
Name: Hour Price, Length: 1160, dtype: float64

In [26]:
def checker(rec):
    return np.mean(pd.Series(rec).dropna())
#     return list(np.unique(rec))
#     return len(np.unique(rec))

In [27]:
df_lawer_assignment_price = content[['Responsible', 'Hour Price', 'Assignment Type']].groupby(['Responsible', 'Assignment Type']).agg(checker)
df_lawer_price = content[['Responsible', 'Hour Price']].groupby(['Responsible']).agg(checker)
df_assignment_price = content[['Assignment Type', 'Hour Price']].groupby(['Assignment Type']).agg(checker)

In [46]:
df_lawer_assignment_price

Hour Price
Responsible       Assignment Type                                     
Boris Keilaniemi  Arviointipalvelu - Asuinhuoneistot        170.000000
                  Arviointipalvelu - Tontit                 100.000000
                  Edunvalvontapalvelu - Korvausneuvottelut  170.000000
                  Edunvalvontapalvelu - Maantiet            170.000000
                  Hallintoasia                              153.478261
                  Kilpailuoikeus                            170.000000
                  Konsultointia                             180.000000
                  Perintöoikeus                             170.000000
                  Rahoitusoikeus                            137.142857
                  Rikosoikeus                               380.000000
                  Sopimusoikeus                             196.666667
                  Velkomusasia                              210.000000
Elias Muonio      Hallintoasia                              131.750000
Helen Hakaniemi   Arviointipalvelu - Tontit                 250.000000
                  Hallintoasia                              132.758621
                  Kilpailuoikeus                            168.175647
                  Konsultointia                             177.500000
                  Perheoikeus                               231.250000
                  Perintöoikeus                             147.096774
                  Rahoitusoikeus                            151.250000
                  Rikosoikeus                               255.220126
                  Sopimusoikeus                             155.000000
                  Työoikeus                                 160.625000
                  Vero-oikeus                               170.000000
                  Yritysoikeus                              157.857143
Jari London       Hallintoasia                              150.000000
                  Konsultointia                             172.000000
                  Perintöoikeus                             185.000000
Jopi Jalkapuoli   Arviointipalvelu - Tontit                 170.000000
                  Hallintoasia                              167.142857
...                                                                ...
Nelli Niittykumpu Edunvalvontapalvelu - Maantiet            300.000000
                  Hallintoasia                              185.375000
                  Jatkuva luottopäätös                      170.000000
                  Kilpailuoikeus                            168.000000
                  Konsultointia                             136.000000
                  Perheoikeus                               166.666667
                  Rahoitusoikeus                            170.000000
                  Rikosoikeus                               176.000000
                  Sopimusoikeus                             190.000000
                  Työoikeus                                 120.000000
                  Velkomusasia                              132.857143
                  Yritysoikeus                              170.000000
Seppo Sörnäinen   Hallintoasia                              182.500000
                  Perheoikeus                               170.000000
Viivi Vuosaari    Arviointipalvelu - Asuinhuoneistot        670.000000
                  Arviointipalvelu - Tontit                 168.000000
                  Hallintoasia                              163.750000
                  Kilpailuoikeus                            159.448182
                  Konsultointia                             176.000000
                  Perheoikeus                               162.000000
                  Perintöoikeus                             170.000000
                  Rahoitusoikeus                            172.500000
                  Rikosoikeus                               147.857143
                  Sopimusoikeus                             185.000000
Yki Kontula  

In [45]:
df_lawer_assignment_price.to_csv('df_lawer_assignment_price.csv', index = True)
df_lawer_price.to_csv('df_lawer_price.csv', index = True)
df_assignment_price.to_csv('df_assignment_price.csv', index = True)

In [28]:
lawer_counts = content[['Responsible', 'Hour Price', 'Assignment Type']].groupby(['Responsible', 'Assignment Type'], as_index=False).agg(np.count_nonzero)

In [29]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [30]:
init_notebook_mode(connected=True)

In [31]:
import plotly
import plotly.graph_objs

In [32]:
responsible_name = 'Boris Keilaniemi'

In [33]:
assignment_name = 'Rahoitusoikeus'

In [34]:
lawer_counts.loc[lawer_counts['Responsible'] == responsible_name, :]

,Responsible,Assignment Type,Hour Price
0,Boris Keilaniemi,Arviointipalvelu - Asuinhuoneistot,5.0
1,Boris Keilaniemi,Arviointipalvelu - Tontit,7.0
2,Boris Keilaniemi,Edunvalvontapalvelu - Korvausneuvottelut,1.0
3,Boris Keilaniemi,Edunvalvontapalvelu - Maantiet,7.0
4,Boris Keilaniemi,Hallintoasia,30.0
5,Boris Keilaniemi,Kilpailuoikeus,7.0
6,Boris Keilaniemi,Konsultointia,3.0
7,Boris Keilaniemi,Perintöoikeus,3.0
8,Boris Keilaniemi,Rahoitusoikeus,6.0
9,Boris Keilaniemi,Rikosoikeus,10.0


In [35]:
mean_assignment_price = int(df_assignment_price.loc[assignment_name, 'Hour Price'])

In [42]:
lawer_counts.to_csv('lawer_counts.csv', index = False)

In [36]:
import plotly.plotly as py

In [37]:
lawer_counts

def get_lawer_assignment_info(responsible_name, assignment_name):
    mean_lawer_price = int(df_lawer_price.loc[responsible_name, 'Hour Price'])
    mean_assignment_price = int(df_assignment_price.loc[assignment_name, 'Hour Price'])
    mean_lawer_assignment_price = int(df_lawer_assignment_price.loc[(responsible_name, assignment_name), 'Hour Price'])
    
    result = {
        'mean_lawer_price': mean_lawer_price,
        'mean_assignment_price': mean_assignment_price,
        'mean_lawer_assignment_price': mean_lawer_assignment_price,
    }
    
    mask_lawer = lawer_counts['Responsible'] == responsible_name
    assignment_types = lawer_counts.loc[mask_lawer, 'Assignment Type'].values
    hour_prices = lawer_counts.loc[mask_lawer, 'Hour Price'].values

#     filename = f'/Users/smerdov/Junction2017/{responsible_name}.html'
#     filename = '/Users/smerdov/Junction2017/Piechart.html'
    fig = plotly.offline.plot(
        {
            "data": [
                plotly.graph_objs.Pie(labels=assignment_types,values=hour_prices)
            ]
        },
#         filename = filename,
        image = 'jpeg',
    )
    
#     py.image.save_as(fig, filename='a-simple-plot.png')
    
    return result

In [38]:
a = get_lawer_assignment_info(responsible_name, assignment_name)

In [244]:
a

{'mean_assignment_price': 172,
 'mean_lawer_assignment_price': 137,
 'mean_lawer_price': 188}

In [288]:
content.to_csv('content.csv', index = False)

In [20]:
json_1['1']['Info']

{'Assigment name': ' Osakekauppariita',
 'Assignment Type': 'Kilpailuoikeus',
 'Author': 'Helen Hakaniemi',
 'Billable Hours_sum': 24.0,
 'Category': nan,
 'Cost Center': nan,
 'Customer Responsible': 'Helen Hakaniemi',
 'Employee Role': nan,
 'Hours Worked_sum': 24.0,
 'Industry': 'Muu',
 'Internal': 'False',
 'Invoiced_sum': 1360.0,
 'Margin_sum': nan,
 'Owner': 'Helen Hakaniemi',
 'Paid_sum': 1360.01,
 'Parent Company': nan,
 'Parent Company Home Country': nan,
 'Parent Company Industry': nan,
 'Parent Company Responsible': nan,
 'Parent Company Type': nan,
 'Payer': nan,
 'Payer Home Country': nan,
 'Principal': 'AsuntoOy Ankka Linna',
 'Principal Home Country': 'Suomi',
 'Principal Type': 'Prospect',
 'Responsible': 'Boris Keilaniemi',
 'Seller': nan,
 'Total_sum': 1360.0}

In [21]:
# def check_count(rec):
#     return len(np.unique(rec))

# agg_names_list = [
#     'Principal',
#     'Customer Responsible',
#     'Owner',
#     'Responsible',
#     'Seller',
#     'Author',
#     'Payer',
# ]

# agg_dict = {name: check_count for name in agg_names_list}

In [24]:
import json

In [25]:
json.dump(json_1, open('json_1', 'w'))

In [7]:
for colname in df.columns:
    # print(df['Principal / Toimeksiantaja'].value_counts()()
    value_counts = df[colname].value_counts()
    print(colname, '\n', 'n different values is', len(value_counts), '\nmost popular is\n', value_counts.head(10), '\n number of nans is', df[colname].isnull().sum())
    print()

Parent Company 
 n different values is 6 
most popular is
 Kolmio Oy                494
Ankallispankki Oy         58
Tiinan Muonitus Oy        53
Leivolan Leivokset Oy      9
Hannu Hanhi Oy             4
A yhtiö Oy                 2
Name: Parent Company, dtype: int64 
 number of nans is 4617

Parent Company Type 
 n different values is 2 
most popular is
 Asiakas       547
Osakeyhtiö     62
Name: Parent Company Type, dtype: int64 
 number of nans is 4628

Parent Company Home Country 
 n different values is 1 
most popular is
 Suomi    620
Name: Parent Company Home Country, dtype: int64 
 number of nans is 4617

Parent Company Industry 
 n different values is 1 
most popular is
 Pankki    60
Name: Parent Company Industry, dtype: int64 
 number of nans is 5177

Parent Company Responsible 
 n different values is 5 
most popular is
 Nelli Niittykumpu     494
Helen Hakaniemi        58
Viivi Vuosaari         53
Lenita Lauttasaari      9
Kustaa Kulosaari        6
Name: Parent Company Responsi